In [385]:
import sys
sys.path.append('..')

from e20009_phases.PointcloudLegacyPhase import PointcloudLegacyPhase, PointCloud, GetLegacyEvent, get_event_range
from e20009_phases.config import ICParameters, DetectorParameters, PadParameters
from spyral.trace.get_legacy_event import (
    GET_DATA_TRACE_START,
    GET_DATA_TRACE_STOP,
)
from spyral.core.run_stacks import form_run_string
from spyral.correction import create_electron_corrector
from spyral.core.pad_map import PadMap
from spyral.core.legacy_beam_pads import LEGACY_BEAM_PADS
from spyral import GetParameters, INVALID_PATH

import h5py as h5
import numpy.random as random
import numpy as np
import polars as pl
import plotly.graph_objects as go
from pathlib import Path
from plotly.subplots import make_subplots
from scipy import signal

def find_trace_from_padid(event: GetLegacyEvent, pad_id: int) -> int:
    for idx, trace in enumerate(event.traces):
        if trace.get_pad_id() == pad_id:
            return idx
    return -1

In [386]:
# Load config
trace_path = Path("/Volumes/e20009/h5")
workspace_path = Path("/Volumes/e20009/e20009_analysis")

pad_params = PadParameters(
    is_default=False,
    is_default_legacy=True,
    pad_geometry_path=Path("/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/pad_geometry_legacy.csv"),
    pad_time_path=Path("/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/pad_time_correction.csv"),
    pad_electronics_path=Path("/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/pad_electronics_legacy.csv"),
    pad_scale_path=Path("/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/pad_electronics_legacy.csv"),
)

get_params = GetParameters(
    baseline_window_scale=20.0,
    peak_separation=5.0,
    peak_prominence=20.0,
    peak_max_width=100.0,
    peak_threshold=30.0,
)

ic_params = ICParameters(
    baseline_window_scale=100.0,
    peak_separation=5.0,
    peak_prominence=30.0,
    peak_max_width=20.0,
    peak_threshold=300.0,
    low_accept=60,
    high_accept=411
)

det_params = DetectorParameters(
    magnetic_field=3.0,
    electric_field=60000.0,
    detector_length=1000.0,
    drift_velocity_path=Path("/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/drift_velocity.csv"),
    get_frequency=3.125,
    garfield_file_path=Path("/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/e20009_efield_correction.txt"),
    do_garfield_correction=False,
)

In [387]:
# Load data
run_number = 350
trace_file_path = trace_path / f"{form_run_string(run_number)}.h5"
trace_file = h5.File(trace_file_path, "r")

trace_group: h5.Group = trace_file['get']

In [388]:
# Ask the trace file for the range of events
min_event, max_event = get_event_range(trace_file)
rng = random.default_rng()
# Select a random event
event_number = rng.integers(min_event, max_event)
# event_number = 304580
print(f'Event: {event_number}')

event_data: h5.Dataset = trace_group[f'evt{event_number}_data']
event = None
correction_path: Path
# Load either a legacy or contemporary GET daq event, and create our assets
event = GetLegacyEvent(event_data, event_number, get_params, ic_params, rng)
phase = PointcloudLegacyPhase(get_params, ic_params, det_params, pad_params)
phase.create_assets(workspace_path)
correction_path = phase.electron_correction_path

pad_map = PadMap(pad_params)

Event: 296523


In [389]:
# #Find downscale beam event
# while event.beam_ds_trace.get_number_of_peaks() == 0:
#     event_number = rng.integers(min_event, max_event)
#     event_data: h5.Dataset = trace_group[f'evt{event_number}_data']
#     event = GetLegacyEvent(event_data, event_number, get_params, ic_params, rng)

# print(event_number)

In [399]:
# Plot trace from event
trace_number = -1
while trace_number == -1:
    random_trace = random.randint(0, len(event_data))
    trace_number = find_trace_from_padid(event, event_data[random_trace, 4])

# pad = 6425
# random_pad = -1
# counter = -1
# while random_pad != pad:
#     counter += 1
#     random_trace = counter
#     random_pad = event_data[counter, 4]
# trace_number = find_trace_from_padid(event, pad)

raw_trace_data = event_data[random_trace]
time_bucket_range = np.arange(start=0, stop=512)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=raw_trace_data[GET_DATA_TRACE_START:GET_DATA_TRACE_STOP], mode="lines", name=f"Raw Trace {trace_number}")
)
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=event.traces[trace_number].trace, mode="lines",name=f"Baseline Corrected Trace {trace_number}")
)
print(f"Trace Number: {trace_number}")
print(f"Trace Hardware: {event.traces[trace_number].hw_id}")
peak_amps = []
peak_cents = []
peak_left = []
peak_left_amps = []
peak_right = []
peak_right_amps = []
for peak in event.traces[trace_number].get_peaks():
    peak_amps.append(peak.amplitude)
    peak_cents.append(np.floor(peak.centroid))
    peak_left.append(peak.positive_inflection)
    peak_right.append(peak.negative_inflection)
    peak_left_amps.append(event.traces[trace_number].trace[int(peak.positive_inflection)])
    peak_right_amps.append(event.traces[trace_number].trace[int(peak.negative_inflection)])
print(f"Peak centroids: {peak_cents}")
fig.add_trace(
    go.Scatter(x=peak_cents, y=peak_amps, mode="markers", name="Peaks")
)
fig.add_trace(
    go.Scatter(x=peak_left, y=peak_left_amps, mode="markers", name="Peak Left Edges")
)
fig.add_trace(
    go.Scatter(x=peak_right, y=peak_right_amps, mode="markers", name="Peak Right Edges")
)
fig.update_legends()
fig.update_layout(
    xaxis_title="Time Bucket",
    yaxis_title="Amplitude",
    showlegend=True
)
fig.show()

Trace Number: 38
Trace Hardware: HardwareID -> pad: 939 cobo: 2 asad: 0 aget: 1 channel: 0
Peak centroids: [84.0]


In [391]:
# Plot DS beam trace
ds_trace = event.beam_ds_trace
print(ds_trace.get_number_of_peaks())

print(f"Beam downscale number of peaks: {ds_trace.get_number_of_peaks()}")

for trace in event_data:
    #Get raw IC SCA trace
    if (trace[0] == 10 and
        trace[2] == 3 and
        trace[3] == 34
        ):
        raw_ds_data = trace

time_bucket_range = np.arange(start=0, stop=512)
hover_text = [f"Trace {i}<br>Peak {j}" for i, t in enumerate(event.traces) for j, _ in enumerate(t.peaks)] # We'll use this later

fig = go.Figure()

#Raw Beam DS trace
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=raw_ds_data[GET_DATA_TRACE_START:GET_DATA_TRACE_STOP], mode="lines", name="Raw Beam DS Trace")
)

#Baseline corrected beam DS trace
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=ds_trace.trace, mode="lines",name="Baseline Corrected Beam DS Trace")
)

peak_amps = []
peak_cents = []
peak_left = []
peak_left_amps = []
peak_right = []
peak_right_amps = []
for peak in ds_trace.get_peaks():
    peak_amps.append(peak.amplitude)
    peak_cents.append(peak.centroid)
    peak_left.append(peak.positive_inflection)
    peak_right.append(peak.negative_inflection)
    peak_left_amps.append(ds_trace.trace[int(peak.positive_inflection)])
    peak_right_amps.append(ds_trace.trace[int(peak.negative_inflection)])
fig.add_trace(
    go.Scatter(x=peak_cents, y=peak_amps, mode="markers", name="Peaks")
)
fig.add_trace(
    go.Scatter(x=peak_left, y=peak_left_amps, mode="markers", name="Peak Left Edges")
)
fig.add_trace(
    go.Scatter(x=peak_right, y=peak_right_amps, mode="markers", name="Peak Right Edges")
)
fig.update_legends()
fig.update_layout(
    xaxis_title="Time Bucket",
    yaxis_title="Amplitude",
    showlegend=True
)
fig.show()

0
Beam downscale number of peaks: 0


In [392]:
# # Make reconstructed mesh signal
# mesh = np.zeros(512)
# for trace in event_data:
#     if trace[4] in LEGACY_BEAM_PADS:
#         mesh += trace[GET_DATA_TRACE_START:GET_DATA_TRACE_STOP]

# # Create the filter
# window = np.arange(-256.0, 256.0, 1.0)
# fil = np.fft.ifftshift(np.sinc(window / 80))
# transformed = np.fft.fft2(mesh, axes=(0,))
# result = np.real(
#     np.fft.ifft2(transformed * fil, axes=(0,))
#     )  # Apply the filter -> multiply in Fourier = convolve in normal

# # for trace in event.traces:
# #     if trace.get_pad_id() in LEGACY_BEAM_PADS:
# #         mesh += trace.trace

# pks, props = signal.find_peaks(
#     result,
#     distance=400,
#     prominence=300,
#     width=(300, 400),
#     rel_height=0.90
#     )

# print(pks)
# print(props)

# #Plot it
# time_bucket_range = np.arange(start=0, stop=512)

# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(x=time_bucket_range, y=mesh, mode="lines", name="Reconstructed Mesh Signal")
# )
# fig.add_trace(
#     go.Scatter(x=time_bucket_range, y=result, mode="lines", name="Smoothed reconstructed Mesh Signal")
# )
# fig.update_legends()
# fig.update_layout(
#     xaxis_title="Time Bucket",
#     yaxis_title="Amplitude",
#     showlegend=True
# )
# fig.show()

In [393]:
# Plot IC and IC SCA traces
ic_trace = event.ic_trace
ic_sca_trace = event.ic_sca_trace

print(f"IC SCA number of peaks: {ic_sca_trace.get_number_of_peaks()}")
print(f"IC number of peaks: {ic_trace.get_number_of_peaks()}")
print(ic_trace.get_peaks())

for trace in event_data:
    #Get raw IC trace
    if (trace[0] == ic_trace.hw_id.cobo_id and
        trace[1] == ic_trace.hw_id.asad_id and
        trace[2] == ic_trace.hw_id.aget_id and
        trace[3] == ic_trace.hw_id.aget_channel and
        trace[4] == ic_trace.hw_id.pad_id
        ):
        raw_ic_data = trace

    #Get raw IC SCA trace
    if (trace[0] == 10 and
        trace[2] == 2 and
        trace[3] == 34
        ):
        raw_ic_sca_data = trace

time_bucket_range = np.arange(start=0, stop=512)
hover_text = [f"Trace {i}<br>Peak {j}" for i, t in enumerate(event.traces) for j, _ in enumerate(t.peaks)] # We'll use this later

fig = go.Figure()

#Raw IC SCA trace
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=raw_ic_sca_data[GET_DATA_TRACE_START:GET_DATA_TRACE_STOP], mode="lines", name=f"Raw IC SCA Trace {trace_number}")
)

#Baseline corrected IC SCA trace
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=ic_sca_trace.trace, mode="lines",name=f"Baseline Corrected IC SCA Trace")
)

#Raw IC trace
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=raw_ic_data[GET_DATA_TRACE_START:GET_DATA_TRACE_STOP], mode="lines", name=f"Raw IC trace {trace_number}")
)

#Baseline corrected IC trace
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=ic_trace.trace, mode="lines",name=f"Baseline Corrected IC Trace")
)

print(f"Trace Hardware: {event.traces[trace_number].hw_id}")
peak_amps = []
peak_cents = []
peak_left = []
peak_left_amps = []
peak_right = []
peak_right_amps = []
for peak in ic_trace.get_peaks():
    peak_amps.append(peak.amplitude)
    peak_cents.append(peak.centroid)
    peak_left.append(peak.positive_inflection)
    peak_right.append(peak.negative_inflection)
    peak_left_amps.append(event.traces[trace_number].trace[int(peak.positive_inflection)])
    peak_right_amps.append(event.traces[trace_number].trace[int(peak.negative_inflection)])
fig.add_trace(
    go.Scatter(x=peak_cents, y=peak_amps, mode="markers", name="Peaks")
)
fig.add_trace(
    go.Scatter(x=peak_left, y=peak_left_amps, mode="markers", name="Peak Left Edges")
)
fig.add_trace(
    go.Scatter(x=peak_right, y=peak_right_amps, mode="markers", name="Peak Right Edges")
)
fig.update_legends()
fig.update_layout(
    xaxis_title="Time Bucket",
    yaxis_title="Amplitude",
    showlegend=True
)
fig.show()

IC SCA number of peaks: 0
IC number of peaks: 0
[]
Trace Hardware: HardwareID -> pad: 8927 cobo: 6 asad: 2 aget: 0 channel: 0


In [394]:
# Make point cloud
cloud = PointCloud()

# Do the electric field correction if requested
corrector = None
if correction_path.exists():
    corrector = create_electron_corrector(correction_path)

cloud.load_cloud_from_get_event(event, pad_map)
hover_text = [f"Pad ID: {int(point[5])}" for point in cloud.cloud] # We'll use this later

fig = make_subplots(2, 1, row_heights=[0.66, 0.33], specs=[[{"type": "xy"}], [{"type": "scene"}]])
fig.add_trace(
    go.Scatter3d(
        x=cloud.cloud[:, 2], 
        y=cloud.cloud[:, 0], 
        z=cloud.cloud[:, 1], 
        mode="markers",
        text = hover_text,
        hovertemplate="X: %{y:.2f}<br>Y: %{z:.2f}<br>Z: %{x:.2f}<br>%{text}",
        marker= {
            "size": 3, 
            "color": cloud.cloud[:, 3], 
            "showscale": True
            }, 
        name="Point Cloud"
    ),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=cloud.cloud[:, 0], 
        y=cloud.cloud[:, 1], 
        mode="markers",
        text = hover_text,
        hovertemplate="X: %{x:.2f}<br>Y: %{y:.2f}<br>%{text}",
        marker= {
            "color": cloud.cloud[:, 3], 
            "showscale": True
        }, 
        name="XY Projection"),
    row=1,
    col=1
)
fig.update_layout(
    xaxis_title = "X (mm)",
    yaxis_title = "Y (mm)",
    xaxis_range=[-300.0, 300.0],
    yaxis_range=[-300.0, 300.0],
    scene = {
        "xaxis_title": "Z (Time Buckets)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        },
        "xaxis_range": [0.0, 512.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
    },
    width = 1000,
    height = 1500,
    showlegend=False
)
fig.show()

In [395]:
# Plot z-calibrated cloud
dv_lf: pl.LazyFrame = pl.scan_csv(det_params.drift_velocity_path)
dv_df: pl.DataFrame = dv_lf.filter(pl.col("run") == run_number).collect()
mm_tb: float = dv_df.get_column("micro_mean")[0]
w_tb: float = dv_df.get_column("wind_mean")[0]

cloud.calibrate_z_position(mm_tb, w_tb, det_params.detector_length, efield_correction=corrector)
fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x=cloud.cloud[:, 2], 
        y=cloud.cloud[:, 0], 
        z=cloud.cloud[:, 1], 
        mode="markers", 
        marker= {
            "size": 3, 
            "color": cloud.cloud[:, 4], 
            "showscale": True
        }, 
        name="Point Cloud"
    )
)
fig.update_layout(
    scene = {
        "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    height=750,
)
fig.show()

In [396]:
# Plot r-Charge projection
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.linalg.norm(cloud.cloud[:, :3], axis=1), y=cloud.cloud[:, 4], mode="markers", marker={"size": 5})
)
fig.update_layout(
    xaxis_title="Position (mm)",
    yaxis_title="Integral"
)
fig.show()